## Advent of Code 2021 - Day 16

### Star 1

In [1]:
import numpy as np

In [2]:
with open('input_16_a.txt') as f:
    tx = f.readline().strip()
tx_bin = [bin(int(x, 16))[2:].zfill(4) for x in tx]
tx_bin = ''.join(tx_bin)

In [3]:
def bin2int(string):
    return int('0b0' + ''.join(string), 2)

In [4]:
def parse_lit_val(tx_bin):
    value = []
    idx = 6
    last = False
    while not last:
        value.append(tx_bin[idx+1:idx+5])
        last = tx_bin[idx] == '0'
        idx += 5
    value = bin2int(value)

    return (value, tx_bin[idx:])

In [5]:
def parse_header(tx_bin):
    ver_id = bin2int(tx_bin[0:3])
    type_id = bin2int(tx_bin[3:6])
    operator = type_id != 4
    num_subpacks = -1
    num_subbits = -1
    lit_val = -1
    if operator:
        if tx_bin[6] == '0':
            num_subbits = bin2int(tx_bin[7:22])
            remain_tx = tx_bin[22:]
        else:
            num_subpacks = bin2int(tx_bin[7:18])
            remain_tx = tx_bin[18:]
    else:
        lit_val, remain_tx = parse_lit_val(tx_bin)       
    return {
        'ver_id': ver_id,
        'type_id': type_id,
        'lit_val': lit_val,
        'operator': operator,
        'num_subpacks': num_subpacks,
        'num_subbits': num_subbits,
        'remain_tx': remain_tx,
    }

In [6]:
def parse(tx_bin, num_subpacks, num_subbits):
    tx_bin_len = len(tx_bin)
    ver_sum = 0
    num_bits = 0
    while not (num_subpacks == 0 or num_subbits == num_bits):
        p_dict = parse_header(tx_bin)
        if p_dict['operator']:
            (tx_bin, v) = parse(p_dict['remain_tx'],
                                      p_dict['num_subpacks'],
                                      p_dict['num_subbits'])
            ver_sum += v + p_dict['ver_id']
        else:
            tx_bin = p_dict['remain_tx']
            ver_sum += p_dict['ver_id']
        num_subpacks -= 1
        num_bits = tx_bin_len - len(tx_bin)
    return (tx_bin, ver_sum)

In [7]:
print(f'Version ID sum = {parse(tx_bin, 1, -1)[1]}')

Version ID sum = 979


### Star 2

In [8]:
def instruction(operator, operands):
    if operator == 4:
        return operands[0]
    elif operator == 0:
        return sum(operands)
    elif operator == 1:
        return int(np.prod(operands))
    elif operator == 2:
        return min(operands)
    elif operator == 3:
        return max(operands)
    elif operator == 5:
        return 1 if operands[0] > operands[1] else 0
    elif operator == 6:
        return 1 if operands[0] < operands[1] else 0
    elif operator == 7:
        return 1 if operands[0] == operands[1] else 0

In [9]:
def parse(tx_bin, num_subpacks, num_subbits, operator):
    tx_bin_len = len(tx_bin)
    num_bits = 0
    operands = []
    while not (num_subpacks == 0 or num_subbits == num_bits):
        p_dict = parse_header(tx_bin)
        if p_dict['operator']:
            (tx_bin, val) = parse(p_dict['remain_tx'],
                                 p_dict['num_subpacks'],
                                 p_dict['num_subbits'],
                                 p_dict['type_id'])
            operands.append(val)
        else:
            tx_bin = p_dict['remain_tx']      
            operands.append(p_dict['lit_val'])
        num_subpacks -= 1
        num_bits = tx_bin_len - len(tx_bin)
    
    return (tx_bin, instruction(operator, operands))

In [10]:
answer = parse(tx_bin, 1, -1, 4)
print(f'Answer = {answer[1]}')

Answer = 277110354175
